取出 acc:0 的 item

In [6]:
import json

def load_json_without_comments(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    content = "\n".join(line for line in lines if not line.lstrip().startswith("//"))
    return json.loads(content)

def filter_acc_zero(data):
    # 遍歷字典中所有 key, 若 value 為 list 則濾出 "acc": 0 的 item
    if isinstance(data, dict):
        for key, value in data.items():
            if isinstance(value, list):
                data[key] = [item for item in value if not (isinstance(item, dict) and item.get("acc") != 0)]
            else:
                filter_acc_zero(value)
    elif isinstance(data, list):
        for item in data:
            filter_acc_zero(item)
    return data

if __name__ == "__main__":
    input_path = "/home/han/Documents/repos/ShowUI/OmniParser/screenspot_omniparser_tmp_dict.json"
    output_path = "/home/han/Documents/repos/ShowUI/OmniParser/screenspot_omniparser_tmp_dict_filter_0.json"

    # 讀取並解析 JSON (移除註解)
    data = load_json_without_comments(input_path)

    # 篩選出 acc 為 0 的項目: 若 item 有 acc 欄位且不是 0，則捨棄
    filtered_data = filter_acc_zero(data)

    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(filtered_data, f, ensure_ascii=False, indent=4)
    
    print("Filtered JSON saved to", output_path)

Filtered JSON saved to /home/han/Documents/repos/ShowUI/OmniParser/screenspot_omniparser_tmp_dict_filter_0.json


隨機抽樣 res

In [ ]:
import json
import random

input_path = "screenspot_omniparser_tmp_dict_filter_0.json"
output_path = "screenspot_omniparser_tmp_dict_sampled.json"

def load_json_without_comments(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    content = "\n".join(line for line in lines if not line.lstrip().startswith("//"))
    return json.loads(content)


data = load_json_without_comments(input_path)
# 指定前兩層 key, 此範例使用 "desktop" 與 "icon"
items = data.get("desktop", {}).get("icon", [])

# 隨機取樣
sample_size = 10
if len(items) < sample_size:
    sampled_items = items
else:
    sampled_items = random.sample(items, sample_size)

print("Sampled items:" + str(sampled_items))
print("length of sampled items: " + str(len(sampled_items)))
    
# # 更新指定路徑的 list
# data["desktop"]["icon"] = sampled_items

# with open(output_path, "w", encoding="utf-8") as f:
#     json.dump(data, f, ensure_ascii=False, indent=4)

# print("Sampled items saved to", output_path)

Sampled items:[{'split': 'desktop', 'data_type': 'icon', 'anno_id': 'screenspot_322', 'img_path': '/home/han/Documents/repos/ShowUI/datasets/ScreenSpot/images/pc_857c0a8b-5f0f-4d6a-966f-789b594622c9.png', 'instruction': 'terminate this process', 'sentence': '[0, 0]', 'pred_content': None, 'bbox': [555, 18, 79, 67], 'meta': {'split': 'desktop', 'id': 'screenspot_322', 'task': 'terminate this process', 'img_url': 'pc_857c0a8b-5f0f-4d6a-966f-789b594622c9.png', 'img_size': [1478, 1288], 'bbox': [555, 18, 79, 67], 'data_type': 'icon', 'data_source': 'macos'}, 'gt_bbox': [0.37550744248985113, 0.013975155279503106, 0.428958051420839, 0.06599378881987578], 'pred_point': [0, 0], 'acc': 0}, {'split': 'desktop', 'data_type': 'icon', 'anno_id': 'screenspot_278', 'img_path': '/home/han/Documents/repos/ShowUI/datasets/ScreenSpot/images/pc_8fc1a1fb-00b9-4fe6-8fd3-4784ab2274e8.png', 'instruction': 'choose the red pen', 'sentence': '[0.0208, 0.0194]', 'pred_content': 'Red', 'bbox': [352, 133, 85, 111],

In [2]:
import json
import random
import os
import importlib
import base64
import io
import utils
importlib.reload(utils)
from util.utils import get_som_labeled_img, check_ocr_box, get_caption_model_processor, get_yolo_model
import torch
from ultralytics import YOLO
from PIL import Image
device = 'cuda'
model_path='weights/icon_detect/model.pt'

input_path = "screenspot_omniparser_tmp_dict_filter_0.json"
# output_path = "screenspot_omniparser_tmp_dict_sampled.json"

som_model = get_yolo_model(model_path)
caption_model_processor = get_caption_model_processor(model_name="florence2", model_name_or_path="weights/icon_caption_florence", device=device)


def save_base64_image(base64_str, save_path):
    image = Image.open(io.BytesIO(base64.b64decode(base64_str)))
    image.save(save_path)


def load_json_without_comments(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    content = "\n".join(line for line in lines if not line.lstrip().startswith("//"))
    return json.loads(content)



### 這裡是主要的程式碼 ###
data = load_json_without_comments(input_path)
# 指定前兩層 key, 此範例使用 "desktop" 與 "icon"
items = data.get("desktop", {}).get("icon", [])

# 隨機取樣
sample_size = 10
if len(items) < sample_size:
    sampled_items = items
else:
    sampled_items = random.sample(items, sample_size)    

results = {}


# 處理每一個 item
for item in sampled_items:
    meta = item.get("meta", {})
    # 使用 img_url 作為 image_path，若需要調整路徑請自行修改
    image_path = item.get("img_path", None)
    if not image_path:
        print("找不到 image 路徑，略過此 item")
        continue
    
    print(f"Processing image: {image_path}")
    
    # config
    image = Image.open(image_path)
    box_overlay_ratio = max(image.size) / 3200
    draw_bbox_config = {
        'text_scale': 0.8 * box_overlay_ratio,
        'text_thickness': max(int(2 * box_overlay_ratio), 1),
        'text_padding': max(int(3 * box_overlay_ratio), 1),
        'thickness': max(int(3 * box_overlay_ratio), 1),
    }
    BOX_TRESHOLD = 0.2
    try:
        # 進行 OCR 與 box 找尋s
        ocr_bbox_rslt, is_goal_filtered = check_ocr_box(
            image_path,
            display_img=False,
            output_bb_format='xyxy',
            goal_filtering=None,
            easyocr_args={'paragraph': False, 'text_threshold': 0.9},
            use_paddleocr=True
        )
        text, ocr_bbox = ocr_bbox_rslt
    
    except Exception as e:
        print(f"Error1 processing {image_path}: {e}")
        continue
    try:
        # 取得模型標註後的圖片、標註座標以及 parsed_content_list
        dino_labled_img, label_coordinates, parsed_content_list = get_som_labeled_img(
            image_path,
            som_model,
            BOX_TRESHOLD=0.2,
            output_coord_in_ratio=True,
            ocr_bbox=ocr_bbox,
            draw_bbox_config=draw_bbox_config,
            caption_model_processor=caption_model_processor,
            ocr_text=text,
            use_local_semantics=True,
            iou_threshold=0.9,
            scale_img=False,
            batch_size=128
        )
        
    except Exception as e:
        print(f"Error2 processing {image_path}: {e}")
        continue
    
    # 建立每個 image 對應的資料夾，使用 meta 的 id 為名稱
    anno_id = meta.get("id", "unknown")
    folder_path = os.path.join("results", anno_id)
    os.makedirs(folder_path, exist_ok=True)
    
    # 儲存標註後的圖片 (base64 解碼後存成 PNG)
    labeled_img_path = os.path.join(folder_path, "labeled.png")
    save_base64_image(dino_labled_img, labeled_img_path)
    
    # 儲存漂亮格式化的 parsed_content_list
    parsed_json_path = os.path.join(folder_path, "parsed_content_list.json")
    with open(parsed_json_path, "w", encoding="utf-8") as f:
        json.dump(parsed_content_list, f, ensure_ascii=False, indent=4)
    
    # 將每個 image 的結果儲存到 results 中
    results[anno_id] = {
        "image_path": image_path,
        "labeled_img": labeled_img_path,
        "pred_content": item.get("pred_content", "not found"),
        "instruction": item.get("instruction", "not found"),
        "parsed_content_list": parsed_content_list,
        "label_coordinates": label_coordinates
    }

# 儲存全部結果到 global_results.json
global_results_path = os.path.join("results", "global_results.json")
os.makedirs("results", exist_ok=True)
with open(global_results_path, "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=4)
    
print("Completed inference and saved results.")

Processing image: /home/han/Documents/repos/ShowUI/datasets/ScreenSpot/images/pc_c8b036c3-da02-490e-a09b-0a42b8d9f344.png

0: 736x1280 29 icons, 5.9ms
Speed: 2.2ms preprocess, 5.9ms inference, 0.5ms postprocess per image at shape (1, 3, 736, 1280)
len(filtered_boxes): 29 18
time to get parsed content: 0.05278968811035156
Processing image: /home/han/Documents/repos/ShowUI/datasets/ScreenSpot/images/pc_4392332d-683f-4746-bbb2-e9287d9cdae0.png

0: 736x1280 13 icons, 5.7ms
Speed: 2.2ms preprocess, 5.7ms inference, 0.5ms postprocess per image at shape (1, 3, 736, 1280)
len(filtered_boxes): 15 8
time to get parsed content: 0.08821749687194824
Processing image: /home/han/Documents/repos/ShowUI/datasets/ScreenSpot/images/pc_33eb9249-4ee4-4752-9541-8466efda3845.png

0: 1024x1280 10 icons, 27.4ms
Speed: 3.6ms preprocess, 27.4ms inference, 0.5ms postprocess per image at shape (1, 3, 1024, 1280)
len(filtered_boxes): 15 9
time to get parsed content: 0.05832552909851074
Processing image: /home/han/D

In [ ]:
import json
import random
import os
import time
import base64
import io
import ast
import matplotlib.pyplot as plt
from PIL import Image

# 假設以下函式與變數已定義，請自行導入或實作：
# check_ocr_box, get_som_labeled_img, draw_bbox_config, caption_model_processor, som_model, BOX_TRESHOLD


def main():
    # 載入經抽樣後的 JSON，包含 "desktop" -> "icon" 的 list
    input_path = "screenspot_omniparser_tmp_dict_sampled.json"
    data = load_json_without_comments(input_path)
    items = data.get("desktop", {}).get("icon", [])
    
    results = {}
    
    # 處理每一個 item
    for item in items:
        meta = item.get("meta", {})
        # 使用 img_url 作為 image_path，若需要調整路徑請自行修改
        image_path = meta.get("img_url", item.get("img_path"))
        if not image_path:
            print("找不到 image 路徑，略過此 item")
            continue
        
        print(f"Processing image: {image_path}")
        try:
            # 進行 OCR 與 box 找尋
            ocr_bbox_rslt, is_goal_filtered = check_ocr_box(
                image_path,
                display_img=False,
                output_bb_format='xyxy',
                goal_filtering=None,
                easyocr_args={'paragraph': False, 'text_threshold': 0.9},
                use_paddleocr=True
            )
            text, ocr_bbox = ocr_bbox_rslt
            cur_time_ocr = time.time()
            
            # 取得模型標註後的圖片、標註座標以及 parsed_content_list
            dino_labled_img, label_coordinates, parsed_content_list = get_som_labeled_img(
                image_path,
                som_model,
                BOX_TRESHOLD=BOX_TRESHOLD,
                output_coord_in_ratio=True,
                ocr_bbox=ocr_bbox,
                draw_bbox_config=draw_bbox_config,
                caption_model_processor=caption_model_processor,
                ocr_text=text,
                use_local_semantics=True,
                iou_threshold=0.7,
                scale_img=False,
                batch_size=128
            )
            cur_time_caption = time.time()
            
        except Exception as e:
            print(f"Error processing {image_path}: {e}")
            continue
        
        # 建立每個 image 對應的資料夾，使用 meta 的 id 為名稱
        anno_id = meta.get("id", "unknown")
        folder_path = os.path.join("results", anno_id)
        os.makedirs(folder_path, exist_ok=True)
        
        # 儲存標註後的圖片 (base64 解碼後存成 PNG)
        labeled_img_path = os.path.join(folder_path, "labeled.png")
        save_base64_image(dino_labled_img, labeled_img_path)
        
        # 儲存漂亮格式化的 parsed_content_list
        parsed_json_path = os.path.join(folder_path, "parsed_content_list.json")
        with open(parsed_json_path, "w", encoding="utf-8") as f:
            json.dump(parsed_content_list, f, ensure_ascii=False, indent=4)
        
        # 將每個 image 的結果儲存到 results 中
        results[anno_id] = {
            "image_path": image_path,
            "labeled_img": labeled_img_path,
            "parsed_content_list": parsed_content_list,
            "label_coordinates": label_coordinates
        }
    
    # 儲存全部結果到 global_results.json
    global_results_path = os.path.join("results", "global_results.json")
    os.makedirs("results", exist_ok=True)
    with open(global_results_path, "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=4)
        
    print("Completed inference and saved results.")
